In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta import add_all_ta_features
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import streamlit as st

In [ ]:
# 1. Data Collection
def load_data(ticker, start, end):
    data = yf.download(ticker, start=start, end=end)
    return data

In [ ]:
# 2. Data Cleaning & Preparation
def prepare_data(data):
    data = data.dropna()
    data = add_lagged_features(data)
    return data

def add_lagged_features(data):
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        for lag in [1, 3, 5]:
            data[f'{col}_lag{lag}'] = data[col].shift(lag)
    return data

In [ ]:
# 3. Feature Engineering
def engineer_features(data):
    # Ensure required columns exist and are not all NaN
    required_cols = ["Open", "High", "Low", "Close", "Volume"]
    for col in required_cols:
        if col not in data.columns or data[col].isnull().all():
            print(f"Column '{col}' is missing or contains only NaN values.")
            return pd.DataFrame()  # Return empty DataFrame to trigger error handling

    # Technical indicators
    data = add_all_ta_features(
        data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True
    )
    # ...existing code...
    data['rolling_mean_5'] = data['Close'].rolling(5).mean()
    data['rolling_std_5'] = data['Close'].rolling(5).std()
    data['return_1'] = data['Close'].pct_change(1)
    data['return_3'] = data['Close'].pct_change(3)
    data['return_5'] = data['Close'].pct_change(5)
    data['volatility'] = data['return_1'].rolling(5).std().shift(-1)
    data = data.dropna()
    return data

In [ ]:
# 4. Train-Test Split
def split_data(data):
    X = data.drop(['volatility'], axis=1)
    y = data['volatility']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    return X_train, X_test, y_train, y_test

In [ ]:
# 5. Modeling
def train_model(X_train, y_train):
    model = XGBRegressor()
    model.fit(X_train, y_train)
    return model

In [ ]:
# 6. Model Evaluation
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    plt.figure(figsize=(10,5))
    plt.plot(y_test.index, y_test, label='Actual')
    plt.plot(y_test.index, y_pred, label='Predicted')
    plt.title('Predicted vs Actual Volatility')
    plt.legend()
    plt.show()
    importance = model.feature_importances_
    return rmse, r2, importance

In [ ]:
# 7. Optional Deployment: Streamlit App
def run_streamlit():
    st.title("Stock Volatility Prediction App")
    ticker = st.text_input("Ticker Symbol (e.g. AAPL)", "AAPL")
    start = st.date_input("Start Date", pd.to_datetime("2015-01-01"))
    end = st.date_input("End Date", pd.to_datetime("2025-08-13"))
    if st.button("Predict Volatility"):
        with st.spinner("Downloading and processing data..."):
            data = load_data(ticker, start, end)
            if data.empty:
                st.error("No data found for the given ticker and date range.")
                return
            data = prepare_data(data)
            data = engineer_features(data)
            if data.empty or len(data) < 30:
                st.error("Not enough data after cleaning and feature engineering. Try a wider date range.")
                return
            X_train, X_test, y_train, y_test = split_data(data)
            model = train_model(X_train, y_train)
            y_pred = model.predict(X_test)
        st.line_chart(pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}, index=y_test.index))
        st.write(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
        st.write(f"R²: {r2_score(y_test, y_pred):.4f}")
        st.bar_chart(pd.Series(model.feature_importances_, index=X_train.columns))

if __name__ == "__main__":
    print("Enter stock ticker, start date (YYYY-MM-DD), and end date (YYYY-MM-DD):")
    ticker = input("Ticker: ").strip().upper()
    start = input("Start Date (YYYY-MM-DD): ").strip()
    end = input("End Date (YYYY-MM-DD): ").strip()
    data = load_data(ticker, start, end)
    if data.empty:
        print("No data found for the given ticker and date range.")
    else:
        data = prepare_data(data)
        data = engineer_features(data)
        if data.empty or len(data) < 30:
            print("Not enough data after cleaning and feature engineering. Try a wider date range.")
        else:
            X_train, X_test, y_train, y_test = split_data(data)
            model = train_model(X_train, y_train)
            rmse, r2, importance = evaluate_model(model, X_test, y_test)
            print(f"RMSE: {rmse:.4f}")
            print(f"R²: {r2:.4f}")

In [ ]:
run_streamlit()